In [1]:
import csv
import random
import math
import operator
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from pandas import *
import numpy as np
from sklearn.metrics import classification_report 
import time
start_time = time.time()

In [2]:
def mahalanobisDistance(instance1, instance2, length , vari):
    distance = 0
    for x in range(length):
        distance += pow((int(instance1[x]) - float(instance2[x]))*vari[x][x], 2)
    return math.sqrt(distance)
 

In [3]:
def getNeighbors(trainingSet, testInstance, vari):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = mahalanobisDistance(testInstance, trainingSet[x], length, vari)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(len(distances)):
        neighbors.append(distances[x][0])
    return neighbors

In [4]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [5]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0


In [7]:
def main():

    cm = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    # prepare data
    trainingSet=[]
    testSet=[]
    data=[]
    mean=[]
    # generate predictions
    predictions=[]
    y_test=[]
    y_pred=[]
    actual=[]
    
    var1 = np.zeros((24,24), dtype=float)
    var2 = np.zeros((24,24), dtype=float)
    var3 = np.zeros((24,24), dtype=float)
    var_cg = np.zeros((24,24), dtype=float)
    var_fri = np.zeros((24,24), dtype=float)
    var_phar = np.zeros((24,24), dtype=float)

    with open(r'C:\Users\Govind\Downloads\train_mean_1.csv', 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            #if random.random() < split:
            mean.append(dataset[x])
            # else:
              #  testSet.append(dataset[x])
    with open(r'C:\Users\Govind\Downloads\02-Govind\group02\dataset\train_data.csv', 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = dataset[x][y]
           # if random.random() < split:
               # trainingSet.append(dataset[x])
            #else:
            data.append(dataset[x])
    with open(r'C:\Users\Govind\Downloads\train_mean_1.csv', 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            #if random.random() < split:
            trainingSet.append(dataset[x])
           # else:
              #  testSet.append(dataset[x])
    with open(r'C:\Users\Govind\Downloads\02-Govind\group02\dataset\test_data.csv', 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
           # if random.random() < split:
               # trainingSet.append(dataset[x])
            #else:
            testSet.append(dataset[x])

    for col in range(0,24):
        for row in range(0,150):
            if row < 50:
                var1[col][col] += pow((float(data[row][col]) - float(mean[0][col])), 2)
                #print(var1[col][col])
            if row >= 50 and row < 100:
                var2[col][col] += pow((float(data[row][col]) - float(mean[1][col])), 2)
            if row > 100:
                var3[col][col] += pow((float(data[row][col]) - float(mean[2][col])), 2)

    #print('var1:')
    #print(var1)
    #print('var2:')
    #print(var2)
    #print('var3:')
    #print(var3)
    myInt =1/ 49
    var_coast = np.multiply(var1, myInt)
    var_industrial = np.multiply(var2, myInt)
    var_pagoda = np.multiply(var3, myInt)
    #print("var1")	
    #print(var_coast)
    var_coast = np.linalg.inv(var_coast) 
    var_industrial = np.linalg.inv(var_industrial) 
    var_pagoda = np.linalg.inv(var_pagoda) 
    #print("varinverse")
    #print(var_atr)

    print('Train set: ' + repr(len(trainingSet)))
    print('Test set: ' + repr(len(testSet)))

    for x in range(len(testSet)):
        if x < 50:
            var = var_coast
        elif x>=50 and x < 100:
            var = var_industrial
        elif x >= 100:
            var = var_pagoda
        neighbors = getNeighbors(trainingSet, testSet[x], var)
        result = getResponse(neighbors)
        predictions.append(result)
        if testSet[x][-1] =='coast':
            if result =='coast':
                cm[0][0] += 1
            elif result =='industrial_area':
                cm[0][1] += 1
            elif result =='pagoda':
                cm[0][2] += 1
        if testSet[x][-1] =='industrial_area':
            if result =='coast':
                cm[1][0] += 1
            elif result =='industrial_area':
                cm[1][1] += 1
            elif result =='pagoda':
                cm[1][2] += 1
        if testSet[x][-1] =='pagoda':
            if result =='coast':
                cm[2][0] += 1
            elif result =='industrial_area':
                cm[2][1] += 1
            elif result =='pagoda':
                cm[2][2] += 1
        y_test.append(testSet[x][-1])
        actual.append(testSet[x][-1])
        y_pred.append(result)
        #print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')
    print('Confusion Matrix:')
    print(DataFrame(cm, columns=["coast", "industrial_area", "pagoda"], index=["coast", "industrial_area", "pagoda"]))
    print(classification_report(y_test, y_pred))
    
    
main()
#print("Time Taken :: --- %s seconds ---" % (time.time() - start_time))  
'''
for i in range(0,149):
		if predictions[i] == "a" and actual[i] == "a":
			cm[0][0] += 1
			#print(cm[0][1])
		elif predictions[i] == "a" and actual[i]== "b":
			cm[0][1] += 1
			#print(cm[0][1])
		elif predictions[i] == "a" and actual[i]== "c":
			cm[0][2] += 1
			#print(cm[0][1])
		elif predictions[i] == "b" and actual[i]== "a":
			cm[1][0] += 1
			#print(cm[0][1])
		elif predictions[i] == "b" and actual[i]== "b":
			cm[1][1] += 1
			#print(cm[0][1])
		elif predictions[i] == "b" and actual[i]== "c":
			cm[1][2] += 1
			#print(cm[0][1])
		elif predictions[i] == "c" and actual[i]== "a":
			cm[2][0] += 1
			#print(cm[0][1])
		elif predictions[i] == "c" and actual[i]== "b":
			cm[2][1] += 1
			#print(cm[0][1])
		elif predictions[i] == "c" and actual[i]== "c":
			cm[2][2] += 1
	print(cm[0][1])
	

'''




Train set: 3
Test set: 150
Accuracy: 55.333333333333336%
Confusion Matrix:
                 coast  industrial_area  pagoda
coast                7               38       5
industrial_area      0               48       2
pagoda               1               21      28
                 precision    recall  f1-score   support

          coast       0.88      0.14      0.24        50
industrial_area       0.45      0.96      0.61        50
         pagoda       0.80      0.56      0.66        50

    avg / total       0.71      0.55      0.50       150



'\nfor i in range(0,149):\n\t\tif predictions[i] == "a" and actual[i] == "a":\n\t\t\tcm[0][0] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "a" and actual[i]== "b":\n\t\t\tcm[0][1] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "a" and actual[i]== "c":\n\t\t\tcm[0][2] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "b" and actual[i]== "a":\n\t\t\tcm[1][0] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "b" and actual[i]== "b":\n\t\t\tcm[1][1] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "b" and actual[i]== "c":\n\t\t\tcm[1][2] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "c" and actual[i]== "a":\n\t\t\tcm[2][0] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "c" and actual[i]== "b":\n\t\t\tcm[2][1] += 1\n\t\t\t#print(cm[0][1])\n\t\telif predictions[i] == "c" and actual[i]== "c":\n\t\t\tcm[2][2] += 1\n\tprint(cm[0][1])\n\t\n\n'